Reading dataset recipes and filtering out useless columns :

In [12]:
import pandas as pd

data=pd.read_csv(r"recipes.csv")

#Filtering columns based on requirement :-
cols_filtered_data=data[["RecipeId", "Name", "AuthorId", "AuthorName", "CookTime", "PrepTime",
        "DatePublished", "RecipeCategory", "AggregatedRating", "ReviewCount", 
        "Calories", "FatContent", "SaturatedFatContent", "CholesterolContent", "SodiumContent", 
        "CarbohydrateContent", "FiberContent", "SugarContent", "ProteinContent", "RecipeServings"]]

Removing Trash rows

In [13]:
cols_filtered_data=cols_filtered_data.drop(index=range(1438, 1488))

Removing columns not having a rating assigned or lesser than 3 ratings :

In [14]:

#Removing columns without rating :-
cols_filtered_data=cols_filtered_data.dropna(subset=["AggregatedRating"])
cols_filtered_data=cols_filtered_data[cols_filtered_data["ReviewCount"]>3]

Choosing diverse rating dishes :

In [15]:
#Choosing mixed rating dishes :-
rating_above_45=cols_filtered_data[cols_filtered_data["AggregatedRating"]>=4.5][0:10000]
rating_below_4=cols_filtered_data[cols_filtered_data["AggregatedRating"]<4.5][0:33000]
cols_filtered_data=pd.concat([rating_above_45,rating_below_4],axis=0)

Changing date-time to only have year :

In [16]:
#Modifying column to store date in required format :- (Only year of publish)
cols_filtered_data["DatePublished"]=cols_filtered_data["DatePublished"].str[0:4]

Replacing blank and wrong data :

In [17]:


#Replacing all blank entries in columns with default data
for i in cols_filtered_data:
    if i not in ["RecipeId", "Name", "AuthorId", "AuthorName","DatePublished", "RecipeCategory","RecipeServings"]:
        cols_filtered_data[i].fillna(0, inplace=True)
    elif i=="RecipeServings":
        cols_filtered_data[i].fillna(-1, inplace=True)   #-1 code for serving parameter not applicable. Eg: items like sauce

Creating function for converting data like 2H2M to 122 mins :

In [18]:
def time_to_mins(strig):
    if strig==0:
        return 0
    else:
        a=0
        b=0
        if strig[1]=="H":
            a=int(strig[0])
        elif len(strig)>2 and strig[2]=="H":
            a=int(strig[0:2])
        if strig[-1]=="M":
            if len(strig)>2 and strig[-3].isdigit():
                b=int(strig[-3:-1])
            else:
                b=int(strig[-2])
        return a*60+b

Converting Prep time and Cook time to minutes

In [19]:
#Converting PrepTime and CookTime column entries to numerical format :-
cols_filtered_data["PrepTime"]=cols_filtered_data["PrepTime"].str.lstrip("PT")
cols_filtered_data["CookTime"]=cols_filtered_data["CookTime"].str.lstrip("PT")
cols_filtered_data["PrepTime"].fillna(0,inplace=True)
cols_filtered_data["CookTime"].fillna(0,inplace=True)
cols_filtered_data["PrepTime"].replace("0S",0,inplace=True)
cols_filtered_data["CookTime"].replace("0S",0,inplace=True)
cols_filtered_data["PrepTime"]=cols_filtered_data["PrepTime"].apply(time_to_mins)
cols_filtered_data["CookTime"]=cols_filtered_data["CookTime"].apply(time_to_mins)

Removing duplicate name entries :

In [20]:

#Removing duplicate items :-
cols_filtered_data.drop_duplicates(subset=["Name"],inplace=True)

Pasting processed dataset into new file :

In [21]:
cols_filtered_data.to_csv(r"Processed_Recipes_Dataset.csv",index=False)

Reading dataset reviews and filtering out useless columns :

In [22]:
import pandas as pd

data=pd.read_csv(r"reviews.csv")

#Filtering columns based on requirement :-
cols_filtered_data=data[["ReviewId","RecipeId", "AuthorId", "AuthorName", "Rating"]]

Ensuring Reviews dataset only has reviews corresponding to filtered recipes :

In [33]:
filter=pd.read_csv(r"Processed_Recipes_Dataset.csv")
print(filter["ReviewCount"].sum())

acceptable_ids=filter["RecipeId"]


cols_filtered_data=cols_filtered_data[cols_filtered_data["RecipeId"].isin(acceptable_ids)]
#find the number of reviews for each recipe and store it in a dictionary
review_count={}
for i in cols_filtered_data["RecipeId"]:
    if i in review_count:
        review_count[i]+=1
    else:
        review_count[i]=1
print(review_count)
#find the sum of values of the dictionary
sum=0
for i in review_count:
    sum+=review_count[i]
print(sum)

#check for which recipe the review count is not equal to the review count in the dataset
for i in filter["RecipeId"]:
    if i not in review_count:
        print(i)

255312.0
{992: 12, 4523: 7, 7435: 43, 44: 22, 5221: 12, 13307: 148, 517: 6, 4684: 40, 278: 15, 3431: 8, 1451: 7, 7478: 21, 5466: 20, 7226: 16, 780: 12, 1078: 125, 7762: 5, 5277: 30, 7205: 4, 5434: 15, 3862: 4, 4397: 4, 1232: 7, 2502: 8, 3490: 4, 100: 2, 3567: 10, 8451: 5, 3704: 7, 153: 48, 3560: 15, 3472: 14, 8445: 10, 4366: 11, 8497: 43, 4165: 20, 1600: 51, 5378: 5, 468: 66, 6576: 3, 4342: 32, 1857: 4, 288: 3, 5297: 62, 2505: 32, 480: 17, 2086: 17, 2521: 8, 7695: 12, 4167: 36, 536: 66, 3470: 325, 4764: 29, 4296: 56, 4734: 8, 5158: 12, 4367: 4, 3561: 13, 5365: 110, 4312: 6, 3360: 11, 4807: 22, 191: 7, 350: 6, 1485: 6, 2718: 5, 4901: 3, 1705: 4, 316: 5, 4213: 13, 3722: 6, 3516: 8, 4038: 6, 6567: 4, 397: 24, 607: 151, 8320: 6, 246: 65, 2985: 4, 5314: 5, 3201: 12, 8468: 39, 3137: 5, 552: 12, 8245: 5, 3448: 23, 8161: 9, 8278: 40, 3525: 5, 6812: 8, 2987: 18, 249: 7, 3596: 196, 7154: 12, 4287: 10, 4725: 11, 4627: 594, 5414: 9, 2636: 8, 7141: 18, 4829: 4, 4560: 16, 809: 51, 665: 8, 7585: 12, 

Pasting Processed data into new file

In [24]:
cols_filtered_data.to_csv(r"Processed_Reviews_Dataset.csv",index=False)